In [ ]:
from blimp.preprocessing.operetta_parse_metadata import (get_image_metadata,
                                                         get_plate_metadata)
from blimp.preprocessing.operetta_to_ome_tiff import (_aggregate_TCZ_metadata,
                                                      _read_images_single_site)

In [ ]:
from pathlib import Path

in_path = Path(
    "/srv/scratch/berrylab/z3532965/operetta_cls/20220720_4i_mAC-POLR2A/mAC-POLR2A_5PhIAA_220708__2022-07-20T16_31_15/Images/"
)

In [ ]:
get_plate_metadata(in_path / "Index.idx.xml", out_file="/srv/scratch/z3532965/tmp.csv")
# get_image_metadata(in_path / "Index.idx.xml", out_file = "/srv/scratch/z3532965/tmp.csv")

In [ ]:
image_metadata = get_image_metadata(in_path / "Index.idx.xml")
image_metadata.query("PlaneID==1 & ChannelID==1")

In [ ]:
image_metadata[["Row", "Col", "StandardFieldID"]].drop_duplicates()

In [ ]:
from blimp.preprocessing.operetta_to_ome_tiff import _get_metadata_batch

_get_metadata_batch(image_metadata, 0, 120)

In [ ]:
from pathlib import Path

in_path = Path(
    "/srv/scratch/berrylab/z3532965/operetta_cls/20220515_LiveCellOptim/40x_eGFP_400ms_100pc_Cy5_100ms_50pc_multiwell__2022-06-15T12_39_14/Images"
)
out_path = Path(
    "/srv/scratch/berrylab/z3532965/operetta_cls/20220515_LiveCellOptim/40x_eGFP_400ms_100pc_Cy5_100ms_50pc_multiwell__2022-06-15T12_39_14/OME-TIFF"
)

# operetta_to_ome_tiff(in_path,
#                     out_path,
#                     metadata_file="Index.idx.xml",
#                     n_batches=10,
#                     batch_id=0,
#                     mip=True)

In [ ]:
_aggregate_TCZ_metadata(image_metadata, mip=True).query(
    "Row==" + str(4) + " & Col==" + str(3) + "& StandardFieldID==" + str(1)
).URL

In [ ]:
# get average absolute time for each unique imaging site
average_AbsTime = image_metadata.groupby(["Row", "Col", "FieldID", "TimepointID"])[
    "AbsTime"
].mean()
image_metadata_no_z = image_metadata.query("PlaneID==1 & ChannelID==1").drop(
    labels=[
        "ChannelID",
        "ChannelName",
        "ChannelType",
        "MainExcitationWavelength",
        "MainEmissionWavelength",
        "AbsTime",
        "AbsPositionZ",
        "PositionZ",
        "PlaneID",
    ],
    axis=1,
)

In [ ]:
n_z_planes = image_metadata.groupby(["Row", "Col", "FieldID", "TimepointID"])[
    "id"
].count()
n_z_planes.name = "nZ"
n_channels = image_metadata.groupby(["Row", "Col", "FieldID", "TimepointID"])[
    "ChannelID"
].max()
n_channels.name = "nC"

In [ ]:
n_z_planes.shape

In [ ]:
image_metadata_no_z.merge(average_AbsTime, on=["Row", "Col", "FieldID", "TimepointID"])

In [ ]:
import pandas as pd

pd.merge(
    image_metadata_no_z,
    average_AbsTime,
    on=["Row", "Col", "FieldID", "TimepointID"],
    how="left",
)

In [ ]:
OPERETTA_REGEX = r"r(?P<Row>\d+)c(?P<Col>\d+)f(?P<FieldID>\d+)p(?P<PlaneID>\d+)-ch(?P<ChannelID>\d+)sk(?P<TimepointID>\d+)(?P<End>fk\d+fl\d+)"

In [ ]:
import re

p = re.compile(OPERETTA_REGEX)


def _remove_TCZ_filename(pattern, filename, mip=False):
    m = pattern.match(filename)
    new_filename = (
        "r"
        + m.group("Row")
        + "c"
        + m.group("Col")
        + "f"
        + m.group("FieldID")
        + "-"
        + m.group("End")
    )
    if mip:
        new_filename += "-mip.ome.tiff"
    else:
        new_filename += ".ome.tiff"
    return new_filename


filename = "r07c06f03p01-ch1sk1fk1fl1.tiff"
_remove_TCZ_filename(p, filename, mip=True)

In [ ]:
p = re.compile("(a)b")
m = p.match("ab")
m.group()

In [ ]:
images = _read_images_single_site(in_path, image_metadata, 3, 2, 1, 1, 0)

In [ ]:
import numpy as np

np.concatenate([img.data for img in images], axis=2).max(axis=2, keepdims=True).shape

In [ ]:
import numpy as np

# get unique sites
timepoints = image_metadata[["TimepointID"]].drop_duplicates()
sites = image_metadata[["Row", "Col", "StandardFieldID"]].drop_duplicates()[:1]
channels = image_metadata[["ChannelID"]].drop_duplicates()

for site_index, site in sites.iterrows():
    multi_timepoint_img = []
    for timepoint_index, timepoint in timepoints.iterrows():
        multi_channel_img = []
        for channel_index, channel in channels.iterrows():
            imgs = read_images_single_site(
                in_path,
                image_metadata,
                site.Row,
                site.Col,
                site.StandardFieldID,
                channel.ChannelID,
                timepoint.TimepointID,
            )
            # combine z-planes (axis 2)
            multi_channel_img.append(np.concatenate([img.data for img in imgs], axis=2))
        # combine channels (axis 1)
        multi_timepoint_img.append(np.concatenate(multi_channel_img, axis=1))
    # combine timepoints (axis 0)
    multi_timepoint_img = np.concatenate(multi_timepoint_img, axis=0)

In [ ]:
image_metadata[["ChannelID"]].drop_duplicates()

In [ ]:
sites = image_metadata[["Row", "Col", "StandardFieldID"]].drop_duplicates()

In [ ]:
sites

In [ ]:
multi_timepoint_img.shape

In [ ]:
query = (
    "Row=="
    + str(3)
    + " & Col=="
    + str(3)
    + " & StandardFieldID=="
    + str(1)
    + " & ChannelID=="
    + str(1)
    + " & TimepointID=="
    + str(0)
)

xyz = image_metadata.query(query)[
    ["ImageResolutionX", "ImageResolutionY", "AbsPositionZ"]
].drop_duplicates()

# image_resolution_x = xyz.ImageResolutionX[0]
# image_resolution_y = xyz.ImageResolutionY[0]
# image_resolution_z = (xyz.AbsPositionZ.max() - xyz.AbsPositionZ.min()) / (xyz.shape[0]-1)

In [ ]:
xyz.ImageResolutionX.iloc[1]

In [ ]:
z_resolution = get_zyx_resolution(image_metadata, 3, 3, 1, 1, 0)

In [ ]:
z_resolution

In [ ]:
from aicsimageio.writers import OmeTiffWriter

# List of strings representing the names of the data channels
channel_names = (
    image_metadata[["ChannelID", "ChannelName"]]
    .drop_duplicates()
    .sort_values(by=["ChannelID"])
    .ChannelName.tolist()
)

In [ ]:
zyx = get_zyx_resolution(image_metadata, 3, 3, 1, 1, 0)
OmeTiffWriter.save(
    data=multi_timepoint_img,
    uri="file.ome.tif",
    dim_order="TCZYX",
    channel_names=channel_names,
    physical_pixel_sizes=z_resolution,
    parser="lxml",
)

In [ ]:
[np.uint16, np.uint8, np.uint16, np.uint16, np.uint16]

In [ ]:
zyx = get_zyx_resolution(image_metadata, 3, 3, 1, 1, 0)

In [ ]:
filename_list = (
    image_metadata.sort_values(
        by=["TimepointID", "ChannelID", "Row", "Col", "StandardFieldID", "AbsPositionZ"]
    )
    .query(query)
    .URL.tolist()
)

In [ ]:
from pathlib import Path

# Check stitching using standard field ids
from aicsimageio import AICSImage

mip_path = in_path.parent / "OME-TIFF-MIP"

In [ ]:
image_metadata_mip = _aggregate_TCZ_metadata(image_metadata, mip=True)

In [ ]:
image_metadata_mip.columns

In [ ]:
image_metadata_mip_subset = (
    image_metadata_mip.sort_values(by=["FieldID"])
    .query("Row == 4 & Col == 4")[
        ["Row", "Col", "FieldID", "StandardFieldID", "URL", "PositionX", "PositionY"]
    ]
    .drop_duplicates()
)
image_metadata_mip_subset

In [ ]:
image_files = image_metadata_mip_subset.URL.to_list()
images = [AICSImage(mip_path / f) for f in image_files]
standard_field_id_mapping = image_metadata_mip_subset[
    ["FieldID", "StandardFieldID"]
].drop_duplicates()
field_id_to_standard_field_id = standard_field_id_mapping.set_index(
    "FieldID"
).T.to_dict("index")["StandardFieldID"]
field_id_to_standard_field_id

In [ ]:
import numpy as np

new_order = [int(s) for s in field_id_to_standard_field_id.values()]
images_reorder = [images[i - 1] for i in new_order]
stitched_array = np.vstack(
    [
        np.hstack(
            [
                images_reorder[i + 5 * j].get_image_data("YX", T=0, C=0)
                for i in range(0, 7)
            ]
        )
        for j in range(0, 5)
    ]
)

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(20, 20))
plt.imshow(stitched_array)
plt.show()